In [130]:
# Dependencies
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split

from PIL import Image
from pathlib import Path
from tqdm import tqdm

np.random.seed(0)
%matplotlib inline

In [182]:
# Constants
DATA_PATH = Path('./data')
CATEGORIES = ['no_mask', 'mask']
IMG_SIZE = 100

In [184]:
# Preview
load_img('./data/mask/aidai/0_0_1.jpg', target_size=(IMG_SIZE, IMG_SIZE))

In [193]:
# Data preprocessing
def preprocess_data():
    images = []
    labels = []
    
    for category in CATEGORIES:
        path = DATA_PATH/category

        for directory in tqdm(list(path.iterdir())):
            for imgPath in directory.iterdir():
                img = load_img(imgPath, target_size=(IMG_SIZE, IMG_SIZE))
                img_array = img_to_array(img)
                
                class_num = CATEGORIES.index(category)
                
                images.append(img_array)
                labels.append(class_num)
          
    images = np.array(images)
    labels = np.array(labels)
    
    if np.max(images) > 1: images /= 255
    
    return images, labels

X, y = preprocess_data()

In [195]:
# Data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [211]:
# Model architecture
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    
    Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=X_train.shape[1:]),
    MaxPool2D(pool_size=(2, 2)),

    Flatten(),
    
    Dense(2048, activation='relu'),
    Dense(1028, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
# Model initialization
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
          batch_size=128,
          epochs=3,
          verbose=1,
          validation_data=(X_test, y_test))

In [ ]:
# Model evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("CNN Error: %.2f%%" % (100-score[1]*100))